In [3]:
import jieba
import string
from IPython.display import HTML
import InvertSearch

class InvertSeach2():
    #
    def __init__(self, doc_file):
        '''
        init term, doc_id, doc_list from doc_file
        '''
        self.terms = dict()
        self.doc_id = 0
        self.doc_list = []
        
        #read doc_file
        with open(doc_file, 'r') as f:
            data= f.readlines()
            data = set(data)
            f.close()
        
        #update doc_listselfand doc_id, terms
        for doc in data:
            self.insert(doc)
     
    def extender(self, state, cache, cutflage=0):
        '''
        extend cache to result depending on state and cutflage: 0 cut_for_search, 1 cut
        '''
        result = []
        if state:
            result.append(cache)
        else:
            if cutflage == 0:
                result.extend(list(jieba.cut_for_search(cache)))
            elif cutflage == 1:
                result.extend(list(jieba.cut(cache)))
            else:
                print('Worning! cutflage is only 0 and 1 is available. predefined as 0 cut_for_search')
                result.extend(list(jieba.cut_for_search(cache)))
        return result
    
    def parse_doc(self, doc, cutflage=0):
        '''
        set up Chinese and English identifier
        '''
        result = []
        cache = ''
        e_sign = ['-', ':', '.', '!']
        last_state = ''  
        for c in doc:
            cur_state = c in string.ascii_letters or c.isdigit() or c in e_sign
            if c == ' ':
                result.extend(self.extender(last_state, cache, cutflage))
                result.append(' ');
                cache = ''
                last_state = ''
            else:
                if last_state == cur_state:
                    cache += c
                else:
                    result.extend(self.extender(last_state, cache,cutflage))
                    cache = c
                last_state = cur_state
        if cache:
            result.extend(self.extender(last_state, cache, cutflage))
        return result
    
    #over write insert method
    def insert(self, doc):
        '''
        add doc to original doc list
        '''
        self.doc_list.append(doc)
        for term in self.parse_doc(doc):
            if term in self.terms:
                self.terms[term].add(self.doc_id)
            else:
                self.terms[term] = set([self.doc_id])
        self.doc_id += 1
        return self.doc_id + 1
    
    #over write find method
    def find(self, keywords):
        '''
        find keywords in terms reuturn each term set result
        '''
        result = None
        for word in self.parse_doc(keywords, cutflage=1):
            if result is None:
                result = self.terms.get(word, set())
            else:
                result = result & self.terms.get(word, set())
        if result is None:
            result = set()
        return result
    
    #over write convert method
    def convert(self, keywords):
        '''
        convert keywords into set expression
        '''
        result = ''
        #convet keywords to standard match format
        keywords_ = keywords.split()
        keywords = ' '.join(keywords_)
        word = self.parse_doc(keywords, cutflage=1)
        print('|'.join(word))
        sign = ['(', ')', 'and', 'AND', 'or', 'OR', 'not', 'NOT', '-', ' ']
        id = 0
        num = len(word)
        while id < num:
            if word[id] == '(' or word[id] == ')' or word[id] == ' ':
                result += word[id]
            elif word[id] == 'and' or word[id] == 'AND':
                result += '&'
            elif word[id] == 'or' or word[id] == 'OR':
                result += '|'
            elif word[id] == 'not' or word[id] == 'NOT'or word[id] == '-':
                result += '-'
            elif id + 1 < num and word[id+1] not in(' ', ')'):
                result +=  'self.find("{}")'.format(word[id]) + ' & '
            elif id + 2< num and word[id+2] not in sign:
                result += 'self.find("{}")'.format(word[id]) + ' & ' 'self.find("{}")'.format(word[id+2])
                id += 3
                continue
            else:
                result +=  'self.find("{}")'.format(word[id])
            id += 1
        return result
    
    #over write hightlight method
    def hightlight(self, doc, keywords):
        '''
        hightlight keywords in doc
        '''
        sign = ['(', ')', 'and', 'AND', 'or', 'OR', 'not', 'NOT', '-', ' ']
        for word in self.parse_doc(keywords, cutflage=1):
            if word not in sign:
                doc = doc.replace(word, '<span style="color:red">{}</span>'.format(word))
        return doc

    def search(self, keywords):
        '''
        searh keywords in doc_list by invert search
        '''
        result = []
        result_ = self.convert(keywords)
        print(result_)
        print(eval(result_))
        for id in eval(result_):
            result.append(self.hightlight(self.doc_list[id], keywords))
        return result

In [6]:
searcher = InvertSeach2('titles.txt')

keywords = '中国女排 or 3-0 '
result = searcher.search(keywords)
if result:
    for doc in result:
        display(HTML(doc))
else:
    print('No result.')

中国女排| |or| |3-0
self.find("中国女排") | self.find("3-0")
{274, 323, 6, 143}
